0. Load libraries

In [1]:
import tensorflow as tf
import os
import sys
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

2025-10-16 20:04:44.062711: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 20:04:46.175200: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/thesnakeguy/miniconda3/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-10-16 20:04:52.437045: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You m

1) Load the segmentation model

In [2]:
# Model path
data_root = "/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/Vipera_segmentation_train_dataset/"
model_filename = "Vipera_SegmentationModel_V2_100+50epochs.keras"
model_path = os.path.join(data_root, model_filename)

# Import custom functions from the accessory script
sys.path.append("/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/")
from HelperFuncs_IoU_DiceLoss_CombinedLoss import combined_loss, dice_loss, mean_iou

# Load model
model = load_model(
    model_path,
    custom_objects={
        'combined_loss': combined_loss,
        'dice_loss': dice_loss,
        'mean_iou': mean_iou
    } 
)

print("✅ Model loaded successfully!")
model.summary()
model.output_shape

W0000 00:00:1760637905.467392    1850 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


✅ Model loaded successfully!
Model: "functional_2"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 512, 512,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 512, 512,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (

/home/thesnakeguy/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 30 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


(None, 512, 512, 2)

2. Segmenting and Extracting the Snakes


In [ ]:
# Configuration
image_folder = r"/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/Vipera_segmentation_test_dataset/"
output_folder = os.path.join(image_folder, "Extracted_snakes")

# Create output folder with explicit error handling
os.makedirs(output_folder, exist_ok=True)
print(f"Output folder created/verified: {output_folder}\n")

# Get list of image files
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
image_files = [f for f in os.listdir(image_folder) 
               if os.path.splitext(f)[1].lower() in image_extensions
               and not os.path.isdir(os.path.join(image_folder, f))]

print(f"Found {len(image_files)} images to process\n")

# Process all images in the list and save class coverage
output_csv_path = os.path.join(output_folder,"class_coverage.csv")
with open(output_csv_path, "w") as coverage:
    coverage.write("photo_ID,class_coverage\n")    
    for idx, img_name in enumerate(image_files[:20], 1):
        try:
            img_path = os.path.join(image_folder, img_name)
            
            # Load original image for saving later
            img_original = Image.open(img_path).convert('RGB')
            original_size = (img_original.width, img_original.height)
            img_array = np.array(img_original)
            
            # Load and preprocess image the same way as training data
            img_resized = img_original.resize((512, 512))
            img_input = np.array(img_resized, dtype=np.uint8)
            
            # Add batch dimension
            img_input = np.expand_dims(img_input, axis=0)
            
            print(f"[{idx}] Processing {img_name}...", end=" ")
            
            # Get prediction
            mask_output = model.predict(img_input, verbose=0)
            
            # Apply argmax like in your visualization code
            mask_pred = tf.argmax(mask_output, axis=-1).numpy()
            mask_pred = mask_pred[0]
            
            # Check mask coverage
            mask_percentage = (np.sum(mask_pred == 1) / mask_pred.size) * 100
            print(f"(class 1 coverage: {mask_percentage:.1f}%)", end=" ")

            # 3. Write data using the live file handle
            coverage.write(f"{img_name},{mask_percentage:.2f}\n")
            
            # Resize mask back to original size
            mask_pil = Image.fromarray((mask_pred * 255).astype(np.uint8))
            mask_pil = mask_pil.resize(original_size, Image.NEAREST)
            mask_array = np.array(mask_pil).astype(np.float32) / 255.0
            
            # Apply mask to extract feature
            mask_expanded = np.stack([mask_array] * 3, axis=2)
            extracted_rgb = (img_array * mask_expanded).astype(np.uint8)
            
            # Create RGBA image with transparency
            extracted = np.zeros((original_size[1], original_size[0], 4), dtype=np.uint8)
            extracted[:, :, :3] = extracted_rgb
            extracted[:, :, 3] = (mask_array * 255).astype(np.uint8)
            
            # Save
            output_name = os.path.splitext(img_name)[0] + '.png'
            output_path = os.path.join(output_folder, output_name)
            result_img = Image.fromarray(extracted, 'RGBA')
            result_img.save(output_path)
            
            print(f"✓ Saved")
            
        except Exception as e:
            print(f"ERROR processing {img_name}: {e}")
            continue
            
print(f"\nProcessing complete! Results saved to: {output_folder}")

Output folder created/verified: /mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/Vipera_segmentation_test_dataset/Extracted_snakes

